In [27]:
max_words = 2000
max_len = 20
num_classes = 1

# Training
epochs = 5
batch_size = 512
print_batch_n = 100

In [50]:

# из библиотеки google.colab импортируем класс files
from google.colab import files
 
# создаем объект этого класса, применяем метод .upload()
uploaded = files.upload()

Saving test.csv to test (1).csv
Saving train.csv to train (1).csv


In [51]:
import pandas as pd

df_train = pd.read_csv("train (1).csv")
df_test = pd.read_csv("test (1).csv")

In [99]:
df_train.head()

,id,label,tweet
0,1,0,user father dysfunctional selfish drags kids d...
1,2,0,user user thanks lyft credit cant use cause do...
2,3,0,bihday majesty
3,4,0,model love u take u time urð± ðððð...
4,5,0,factsguide society now motivation


In [98]:
df_test.head()

,id,tweet
0,31963,studiolife aislife requires passion dedication...
1,31964,user white supremacists want everyone see new ...
2,31965,safe ways heal acne altwaystoheal healthy healing
3,31966,hp cursed child book reservations already yes ...
4,31967,3rd bihday amazing hilarious nephew eli ahmir ...


In [54]:
df_train['label'].value_counts()

0    29720
1     2242
Name: label, dtype: int64

In [55]:
!pip install stop-words pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 3.2 MB/s 
     |████████████████████████████████| 8.2 MB 26.3 MB/s 
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32911 sha256=f135836cad90283d4299033d42be56e2bd7667319fbe602f2af34467e423f700
  Stored in directory: /root/.cache/pip/wheels/fb/86/b2/277b10b1ce9f73ce15059bf6975d4547cc4ec3feeb651978e9
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=0d717d7986bec60b2c4399aa9e6125667df114c54b472675daf697aa94586ca5
  Stored in directory: /root/.cache/pip/wheels/72/b0/3f/1d95f96ff986c7dfffe46ce2be4062f38ebd04b506c77c81b9
Successfully built stop-words docopt


In [56]:
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

In [58]:
sw = set(get_stop_words("en"))
sw

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 "can't",
 'cannot',
 'could',
 "couldn't",
 'did',
 "didn't",
 'do',
 'does',
 "doesn't",
 'doing',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 "hadn't",
 'has',
 "hasn't",
 'have',
 "haven't",
 'having',
 'he',
 "he'd",
 "he'll",
 "he's",
 'her',
 'here',
 "here's",
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 "how's",
 'i',
 "i'd",
 "i'll",
 "i'm",
 "i've",
 'if',
 'in',
 'into',
 'is',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 "let's",
 'me',
 'more',
 'most',
 "mustn't",
 'my',
 'myself',
 'no',
 'nor',
 'not',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'ought',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'same',
 "shan't",
 'she',
 "she'd",
 "she'll",
 "she's",
 'should',
 "s

In [59]:
puncts = set(punctuation)
puncts

{'!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 '{',
 '|',
 '}',
 '~'}

In [61]:
morpher = MorphAnalyzer()


def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in puncts)
    txt = txt.lower()
    txt = re.sub("not\s", "not", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

In [63]:
df_train['tweet'].iloc[:1].values

array([' @user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction.   #run'],
      dtype=object)

In [64]:
df_train['tweet'].iloc[:1].apply(preprocess_text).values

array(['user father dysfunctional selfish drags kids dysfunction run'],
      dtype=object)

In [65]:
from tqdm import tqdm 
tqdm.pandas()

df_train['tweet'] = df_train['tweet'].progress_apply(preprocess_text)
df_test['tweet'] = df_test['tweet'].progress_apply(preprocess_text)

100%|██████████| 17197/17197 [00:08<00:00, 1923.35it/s]


In [66]:
train_corpus = " ".join(df_train["tweet"])
train_corpus = train_corpus.lower()

In [67]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

tokens = word_tokenize(train_corpus)
tokens[:5]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


['user', 'father', 'dysfunctional', 'selfish', 'drags']

In [68]:
tokens_filtered = [word for word in tokens if word.isalnum()]

In [69]:
from nltk.probability import FreqDist

dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]  # вычитание 1 для padding
len(tokens_filtered_top)

1999

In [70]:
tokens_filtered_top[:10]

['user', 'love', 'day', 'happy', 'amp', 'just', 'will', 'im', 'u', 'time']

In [71]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}
vocabulary

{'user': 1,
 'love': 2,
 'day': 3,
 'happy': 4,
 'amp': 5,
 'just': 6,
 'will': 7,
 'im': 8,
 'u': 9,
 'time': 10,
 'life': 11,
 'like': 12,
 'today': 13,
 'new': 14,
 'now': 15,
 'positive': 16,
 'thankful': 17,
 'get': 18,
 'people': 19,
 'good': 20,
 'bihday': 21,
 'cant': 22,
 'one': 23,
 'see': 24,
 'can': 25,
 'dont': 26,
 'fathers': 27,
 'smile': 28,
 'go': 29,
 'want': 30,
 'take': 31,
 'work': 32,
 'healthy': 33,
 'got': 34,
 'weekend': 35,
 'fun': 36,
 'make': 37,
 'summer': 38,
 'best': 39,
 'friday': 40,
 'us': 41,
 'way': 42,
 'great': 43,
 'need': 44,
 'bull': 45,
 'family': 46,
 'beautiful': 47,
 '2': 48,
 'friends': 49,
 'days': 50,
 'first': 51,
 'going': 52,
 'really': 53,
 'music': 54,
 'morning': 55,
 'back': 56,
 'world': 57,
 'wait': 58,
 'know': 59,
 'tomorrow': 60,
 'fathersday': 61,
 'never': 62,
 'sad': 63,
 'week': 64,
 'cute': 65,
 'orlando': 66,
 'think': 67,
 'feel': 68,
 'blog': 69,
 'happiness': 70,
 'model': 71,
 'well': 72,
 'right': 73,
 'trump': 74,


In [72]:
import numpy as np


def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])

    padding = [0] * (maxlen-len(result))
    return result[-maxlen:] + padding

In [75]:
%%time
x_train = np.asarray([text_to_sequence(text, max_len) for text in df_train["tweet"]])
x_val = np.asarray([text_to_sequence(text, max_len) for text in df_test["tweet"]])

CPU times: user 10.3 s, sys: 38.7 ms, total: 10.4 s
Wall time: 11.5 s


In [76]:
x_train.shape

(31962, 20)

In [77]:
df_train['tweet'].iloc[0]

'user father dysfunctional selfish drags kids dysfunction run'

In [78]:
x_train[0]

array([  1, 172, 165, 364,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0])

In [79]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self, vocab_size=2000, embedding_dim=128, out_channel=128, num_classes=1):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.conv_1 = nn.Conv1d(embedding_dim, out_channel, kernel_size=2)
        self.conv_2 = nn.Conv1d(embedding_dim, out_channel, kernel_size=3)
        self.pool = nn.MaxPool1d(2)
        self.relu = nn.ReLU()
        self.linear_1 = nn.Linear(out_channel, out_channel // 2)
        self.linear_2 = nn.Linear(out_channel // 2, num_classes)
        
    def forward(self, x):        
        output = self.embedding(x) # B, L, E
        #                       B  E  L         
        output = output.permute(0, 2, 1)
        output = self.conv_1(output)
        output = self.relu(output)
        output = self.pool(output)

        output = self.conv_2(output)
        output = self.relu(output)
        output = self.pool(output)
        output = torch.max(output, axis=2).values
        output = self.linear_1(output)
        output = self.relu(output)
        output = self.linear_2(output)
        output = F.sigmoid(output)
        return output

In [80]:
from torch.utils.data import DataLoader, Dataset


class DataWrapper(Dataset):
    def __init__(self, data, target, transform=None):
        self.data = torch.from_numpy(data).long()
        self.target = torch.from_numpy(target).long()
        self.transform = transform
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]
        
        if self.transform:
            x = self.transform(x)
            
        return x, y
    
    def __len__(self):
        return len(self.data)

In [96]:
train_dataset = DataWrapper(x_train, df_train['label'].values)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


val_loader = DataLoader(x_val, batch_size=8, shuffle=True)


In [92]:
model = Net(vocab_size=max_words)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [93]:
print(model)
print("Parameters:", sum([param.nelement() for param in model.parameters()]))

Net(
  (embedding): Embedding(2000, 128)
  (conv_1): Conv1d(128, 128, kernel_size=(2,), stride=(1,))
  (conv_2): Conv1d(128, 128, kernel_size=(3,), stride=(1,))
  (pool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (relu): ReLU()
  (linear_1): Linear(in_features=128, out_features=64, bias=True)
  (linear_2): Linear(in_features=64, out_features=1, bias=True)
)
Parameters: 346497


In [94]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.BCELoss()

In [103]:
model = model.to(device)
model.train()
th = 0.5

train_loss_history = []
test_loss_history = []


for epoch in range(epochs):  
    running_items, running_right = 0.0, 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        
        # обнуляем градиент
        optimizer.zero_grad()
        outputs = model(inputs)
        
        loss = criterion(outputs, labels.float().view(-1, 1))
        loss.backward()
        optimizer.step()

        # подсчет ошибки на обучении
        loss = loss.item()
        running_items += len(labels)
        # подсчет метрики на обучении
        pred_labels = torch.squeeze((outputs > th).int())
        running_right += (labels == pred_labels).sum()
        
    # выводим статистику о процессе обучения
    model.eval()
    
    print(f'Epoch [{epoch + 1}/{epochs}]. ' \
            f'Step [{i + 1}/{len(train_loader)}]. ' \
            f'Loss: {loss:.3f}. ' \
            f'Acc: {running_right / running_items:.3f}', end='. ')
    print()
    running_loss, running_items, running_right = 0.0, 0.0, 0.0
    train_loss_history.append(loss)

        # выводим статистику на тестовых данных
    test_running_right, test_running_total, test_loss = 0.0, 0.0, 0.0
    # for j, data in enumerate(val_loader):
    #     test_labels = data[1].to(device)
    #     # test_outputs = model(data[0].to(device))
        
    #     # подсчет ошибки на тесте
    #     # test_loss = criterion(test_outputs, test_labels.float().view(-1, 1))
    #     # подсчет метрики на тесте
    #     # test_running_total += len(data[1])
    #     # pred_test_labels = torch.squeeze((test_outputs > th).int())
    #     # test_running_right += (test_labels == pred_test_labels).sum()
    
    # # test_loss_history.append(test_loss.item())
    # # print(f'Test loss: {test_loss:.3f}. Test acc: {test_running_right / test_running_total:.3f}')
    
    model.train()
        
print('Training is finished!')

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1960: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch [1/5]. Step [63/63]. Loss: 0.018. Acc: 0.992. 
Epoch [2/5]. Step [63/63]. Loss: 0.014. Acc: 0.992. 
Epoch [3/5]. Step [63/63]. Loss: 0.027. Acc: 0.992. 
Epoch [4/5]. Step [63/63]. Loss: 0.067. Acc: 0.994. 
Epoch [5/5]. Step [63/63]. Loss: 0.008. Acc: 0.995. 
Training is finished!
